In [1]:
import os
import geopandas as gpd
import numpy as np
from osgeo import gdal
import rasterio
import rasterio.mask
import netCDF4 as nc

In [2]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\base_geojson')
mandal_boundaries = gpd.read_file('TS_mandal_boundary.json')
mandal_boundaries = mandal_boundaries.reset_index()

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\base_geojson')
shape_telangana = gpd.read_file('TL_state_shapefile_for_clip.geojson')
shape_telangana = shape_telangana.explode()

C:\Users\Jesse\AppData\Local\Temp\ipykernel_12208\31479843.py:7: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  shape_telangana = shape_telangana.explode()


In [3]:
# This function rescales the pixel value such that we derive the physical values
def del_rescale(j, nodata_value_new, file_path):
    tiff_open = gdal.Open(file_path) # open the GeoTIFF we want to rescale
    band = tiff_open.GetRasterBand(1) # Select the band
    tiff_array = tiff_open.ReadAsArray() # Assign raster values to a numpy nd array

    nodata_value_old = 255
    nodata_value_old_2 = tiff_open.GetRasterBand(1).GetNoDataValue()

    tiff_array_new = np.where(tiff_array != nodata_value_old, tiff_array, nodata_value_new) # Rescale pixel values and change nodatavalue to -9999
    tiff_array_new = np.where(tiff_array_new != nodata_value_old_2, tiff_array_new, nodata_value_new)

    col = tiff_open.RasterXSize # number of columns
    rows = tiff_open.RasterYSize # number of rows
    driver = tiff_open.GetDriver()

    # Create a new raster to save the average-values
    new_raster = driver.Create(j, col, rows, 1, gdal.GDT_Float32)

    # Copy the properties of the original raster
    new_raster.SetGeoTransform(tiff_open.GetGeoTransform())
    new_raster.SetProjection(tiff_open.GetProjection())

    # Add the average values to newly created raster
    new_raster.GetRasterBand(1).WriteArray(tiff_array_new)
    new_raster.GetRasterBand(1).SetNoDataValue(nodata_value_new)

    # close the raster
    new_raster = None
    del new_raster

In [13]:
# This function rescales the pixel value such that we derive the physical values
def rescale(j, nodata_value_new, Scaling, Offset, file_path):
    tiff_open = gdal.Open(file_path) # open the GeoTIFF we want to rescale
    band = tiff_open.GetRasterBand(1) # Select the band
    tiff_array = tiff_open.ReadAsArray() # Assign raster values to a numpy nd array

    nodata_value_old = tiff_open.GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
    tiff_array_new = np.where(tiff_array != nodata_value_old, tiff_array * Scaling + Offset, nodata_value_new) # Rescale pixel values and change nodatavalue to -9999

    col = tiff_open.RasterXSize # number of columns
    rows = tiff_open.RasterYSize # number of rows
    driver = tiff_open.GetDriver()

    # Create a new raster to save the average-values
    new_raster = driver.Create(j, col, rows, 1, gdal.GDT_Float32)

    # Copy the properties of the original raster
    new_raster.SetGeoTransform(tiff_open.GetGeoTransform())
    new_raster.SetProjection(tiff_open.GetProjection())

    # Add the average values to newly created raster
    new_raster.GetRasterBand(1).WriteArray(tiff_array_new)
    new_raster.GetRasterBand(1).SetNoDataValue(nodata_value_new)

    # close the raster
    new_raster = None
    del new_raster

# Rewriting NETCDF Files
##### Add all NETCDF files that need to be rewritten into geotiff in Input Map

In [5]:
#Choose Parameter

#PM2.5
parameter = ':GWRPM25'
#BA

In [6]:
folder_names = os.listdir('C:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Input\\')
for i in folder_names:
    file_names = os.listdir('C:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Input\\' + i)
    for j in file_names:
        nc_data = "C:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Input\\" + i + "\\" + j
        nc_file = gdal.Open('NETCDF:'+ nc_data + parameter)
        # Convert the netCDF to Geotiff file
        tiff_file = gdal.Translate('C:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\'+ j[:-3] + '.tif', nc_file)

## Cropping Telangana Shape
#### Add all files that need to be cropped into the Output File

In [5]:
#Crop the file in the form of the Telangana Shape file 
file_names = os.listdir("c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output")
for i in file_names:
    file_name = "c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\" + i
    with rasterio.open(file_name) as src:
        out_image, out_transform = rasterio.mask.mask(src, shape_telangana.geometry, crop=True)
        out_meta = src.meta
    
    out_meta.update({"driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform})

    # Save the Geotiff file to Burnt Area in Telangana
    with rasterio.open("c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\" + i, "w", **out_meta) as dest:
        dest.write(out_image)

    print(i)

V5GL02.HybridPM25c_0p10.Asia.201501-201501.tif
V5GL02.HybridPM25c_0p10.Asia.201502-201502.tif
V5GL02.HybridPM25c_0p10.Asia.201503-201503.tif
V5GL02.HybridPM25c_0p10.Asia.201504-201504.tif
V5GL02.HybridPM25c_0p10.Asia.201505-201505.tif
V5GL02.HybridPM25c_0p10.Asia.201506-201506.tif
V5GL02.HybridPM25c_0p10.Asia.201507-201507.tif
V5GL02.HybridPM25c_0p10.Asia.201508-201508.tif
V5GL02.HybridPM25c_0p10.Asia.201509-201509.tif
V5GL02.HybridPM25c_0p10.Asia.201510-201510.tif
V5GL02.HybridPM25c_0p10.Asia.201511-201511.tif
V5GL02.HybridPM25c_0p10.Asia.201512-201512.tif
V5GL02.HybridPM25c_0p10.Asia.201601-201601.tif
V5GL02.HybridPM25c_0p10.Asia.201602-201602.tif
V5GL02.HybridPM25c_0p10.Asia.201603-201603.tif
V5GL02.HybridPM25c_0p10.Asia.201604-201604.tif
V5GL02.HybridPM25c_0p10.Asia.201605-201605.tif
V5GL02.HybridPM25c_0p10.Asia.201606-201606.tif
V5GL02.HybridPM25c_0p10.Asia.201607-201607.tif
V5GL02.HybridPM25c_0p10.Asia.201608-201608.tif
V5GL02.HybridPM25c_0p10.Asia.201609-201609.tif
V5GL02.Hybrid

# Rescale the size if necessary

NO2

In [29]:
#First set all 255 values from NO2 to the following new data values
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Changed_NDV\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        del_rescale(i, ndv, tiff_path)
    except:
        print(i)

#For NO2, we need to rescale from 0-254 to 0-1500
scaling = 1500/254
offset = 0
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Changed_NDV\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Rescaled\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

LST

In [34]:
#First set all 255 values from Soil Temperature to the following new data values
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Changed_NDV\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        del_rescale(i, ndv, tiff_path)
    except:
        print(i)

#For Soil Temperature, we need to rescale from 0-254 to -25 - 45
scaling = 70/254
offset = -25
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Changed_NDV\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Rescaled\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

PM2.5

In [14]:
#For PM2.5 we do not need to rescale but only change the no data value to -9999
scaling = 1
offset = 0
ndv = -9999

path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Preprocessing_Data_Dicra\\Output\\'
file_names = os.listdir(path) # contains all filenames
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Rescaled\\') # Set path
for i in file_names:
    tiff_path = path + i # the selected GeoTIFF path
    try:
        rescale(i, ndv, scaling, offset, tiff_path)
    except:
        print(i)

# COG
##### We need to create a geooptimized tif file. Add the following text in OSGeo4W Shell : 

#for %N in (C:/Users/Jesse/OneDrive/Documenten/Rescaled/*.tiff) DO gdal_translate C:/Users/Jesse/OneDrive/Documenten/Rescaled/%~nN.tiff  C:/Users/Jesse/OneDrive/Documenten/COG/%~nN.tiff -co COMPRESS=LZW -co TILED=YES


#OR
for %N in (C:/Users/Jesse/OneDrive/Documenten/Rescaled/*.tif) DO gdal_translate C:/Users/Jesse/OneDrive/Documenten/Rescaled/%~nN.tif  C:/Users/Jesse/OneDrive/Documenten/COG/%~nN.tif -co COMPRESS=LZW -co TILED=YES
 
